# BEAT-PD SC1 Subject Specific Models Notebook

In [1]:
# Imports
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import pytorch_lightning as pl
from pytorch_lightning.logging import TensorBoardLogger
from tensorboard import notebook
from argparse import Namespace
import torch.nn as nn
import torchvision as tv
import torchvision.transforms.functional as TF
from torch.nn import functional as F
from torchvision import transforms, models
from torch.utils import data
from torch.utils.data import DataLoader
from PIL import Image
import random

In [2]:
# Run .py files
%run -i Dataset_ensemble.py
%run -i Torch_Dataset.py
%run -i BEATPD_LSTM.py
%run -i EvalUtilities.py
%run -i LightningEnsemble.py

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Jonathan\Anaconda3\envs\researchenv\l

plot_ROC and BEATPDscoring commands now loaded.


In [3]:
torch.cuda.empty_cache()

# Identifying Sub Challenge 1 Test Subjects

In [4]:
cis_tst_id = pd.read_csv('Data/BEATPDchallenge/CIS-PD_Test_Data_IDs.csv')

In [5]:
real_tst_id = pd.read_csv('Data/BEATPDchallenge/REAL-PD_Test_Data_IDs.csv')

In [6]:
tst_sc1 = pd.read_csv('Data/BEATPDchallenge/BEAT-PD_SC1_OnOff_Submission_Template.csv')

In [7]:
tst_sc1['subject_id'] = np.nan

In [8]:
for idx1, row1 in tst_sc1.iterrows():
    for idx2, row2 in pd.concat([cis_tst_id,real_tst_id]).reset_index(drop=True).iterrows():
        if row1.measurement_id == row2.measurement_id:
            tst_sc1.subject_id[idx1] = row2.subject_id

LightningEnsemble.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from tensorboard import notebook
C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
 tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Create CIS Dataset

In [11]:
############### CIS config ###############

dataset_name = 'CIS'
label_class = 'on_off'
sort_by = 'subject_id'
train_ts_dir = 'Data/BEATPDchallenge/training_data-cispd/'
train_label_dir = 'Data/BEATPDchallenge/data_labels-cispd/CIS-PD_Training_Data_IDs_Labels.csv'
ancil_ts_dir = 'Data/BEATPDchallenge/ancillary_data-cispd/'
ancil_label_dir = 'Data/BEATPDchallenge/data_labels-cispd/CIS-PD_Ancillary_Data_IDs_Labels.csv'
test_ts_dir='Data/BEATPDchallenge/testing_data-cispd/'
test_data_id_dir='Data/BEATPDchallenge/CIS-PD_Test_Data_IDs.csv'

In [12]:
dataset = Dataset(dataset_name=dataset_name, sort_by=sort_by, label_class = label_class,
                           train_ts_dir=train_ts_dir, train_label_dir=train_label_dir,
                           ancil_ts_dir=ancil_ts_dir, ancil_label_dir=ancil_label_dir,
                           combine_ancil=False,test_ts_dir=test_ts_dir, test_data_id_dir=test_data_id_dir)

Creating Test Dictionary: 100%|██████████████████████████████████████████████████████| 618/618 [00:52<00:00, 11.88it/s]


In [13]:
dataset.data_dict.keys()

dict_keys([1004, 1006, 1007, 1019, 1020, 1023, 1032, 1034, 1038, 1039, 1043, 1044, 1046, 1048, 1049, 1051])

In [14]:
dataset.test_data_dict.keys()

dict_keys([1004, 1006, 1007, 1019, 1020, 1023, 1032, 1034, 1038, 1039, 1043, 1044, 1046, 1048, 1049, 1051])

## Model Settings

In [15]:
model_name = "Final Ensemble v1"
path_to_null_preds = 'Data/BEATPDchallenge/null_preds.csv'
checkpoint_callback = False
early_stop_callback=True
max_epochs=15
gpus=1 # Set to None for CPU training
learning_rate = 0.0001
train_batch_size = 2
val_batch_size = 2

# Subject 1004 Model

In [16]:
subject_id = 1004

In [17]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.58s/it]


In [18]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.12it/s]


## Train Validation Split

In [19]:
dataset.train_validation_split(val_proportion=0.2)

In [20]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 116
Validation size: 28


## Run Training/Validation

In [21]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [22]:
# Start tensorboard
notebook.start(f'--logdir lightning_logs --host localhost --port 2657')

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 11848.

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\subprocess.py:883: ResourceWarning: subprocess 11848 is still running
  ResourceWarning, source=self)


In [23]:
notebook.display(port=2657)

Selecting TensorBoard with logdir lightning_logs (started 0:00:19 ago; port 2657, pid 6392).


In [22]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

In [23]:
# List tensorboard instances
#notebook.list()

In [24]:
#tensorboard kill
#!taskkill /pid 7496 /f

## Make Predictions

In [25]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


In [26]:
# View predictions
tst_sc1.head()

,measurement_id,prediction,subject_id
0,0e80673b-8c90-474f-8388-00fdf25795e4,0.0,1004
1,a7b61d40-9f26-4878-a2d7-bd766b8ae6f5,0.0,1004
2,fc851afd-133c-4e8f-8deb-0079cc442358,3.0,1004
3,90c2c320-1007-47b4-9f8e-cdd37067dfb2,0.0,1004
4,379fd23b-e38f-4214-8a70-7a829004359f,4.0,1004


In [27]:
 tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Subject 1006 Model

In [28]:
subject_id = 1006

In [29]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.77s/it]


In [30]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.37it/s]


## Train Validation Split

In [31]:
dataset.train_validation_split(val_proportion=0.2)

In [32]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 64
Validation size: 16


## Run Training/Validation

In [33]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [34]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [35]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


In [36]:
 tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Subject 1007 Model

In [37]:
torch.cuda.empty_cache()

In [38]:
subject_id = 1007

In [39]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.59s/it]

23 samples with missing labels.
 Labels were replaced with key label mean.


In [40]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.85s/it]


## Train Validation Split

In [41]:
dataset.train_validation_split(val_proportion=0.2)

In [42]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 240
Validation size: 59


## Run Training/Validation

In [43]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [44]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [45]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


In [46]:
 tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Subject 1019 Model

In [47]:
torch.cuda.empty_cache()

In [48]:
subject_id = 1019

In [49]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.91s/it]


In [50]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.63it/s]


## Train Validation Split

In [51]:
dataset.train_validation_split(val_proportion=0.2)

In [52]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 64
Validation size: 16


## Run Training/Validation

In [53]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [54]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [55]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


In [56]:
 tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Subject 1020 Model

In [57]:
torch.cuda.empty_cache()

In [58]:
subject_id = 1020

In [59]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.44s/it]


In [60]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.45s/it]


## Train Validation Split

In [61]:
dataset.train_validation_split(val_proportion=0.2)

In [62]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 156
Validation size: 39


## Run Training/Validation

In [63]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [64]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [65]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


In [66]:
 tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Subject 1023 Model

In [67]:
torch.cuda.empty_cache()

In [68]:
subject_id = 1023

In [69]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.32s/it]


In [70]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.38s/it]


## Train Validation Split

In [71]:
dataset.train_validation_split(val_proportion=0.2)

In [72]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 148
Validation size: 37


## Run Training/Validation

In [73]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [74]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [75]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


In [76]:
 tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Subject 1032 Model

In [77]:
torch.cuda.empty_cache()

In [78]:
subject_id = 1032

In [79]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.10s/it]


In [80]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.20s/it]


## Train Validation Split

In [81]:
dataset.train_validation_split(val_proportion=0.2)

In [82]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 142
Validation size: 35


## Run Training/Validation

In [83]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [84]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [85]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


In [86]:
 tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Subject 1034 Model

In [87]:
torch.cuda.empty_cache()

In [88]:
subject_id = 1034

In [89]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.46s/it]


In [90]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.79it/s]


## Train Validation Split

In [91]:
dataset.train_validation_split(val_proportion=0.2)

In [92]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 52
Validation size: 12


## Run Training/Validation

In [93]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [94]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [95]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


In [96]:
 tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Subject 1038 Model

In [97]:
torch.cuda.empty_cache()

In [98]:
subject_id = 1038

In [99]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.87s/it]


In [100]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.76s/it]


## Train Validation Split

In [101]:
dataset.train_validation_split(val_proportion=0.2)

In [102]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 166
Validation size: 41


## Run Training/Validation

In [103]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [104]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [105]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


In [106]:
 tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Subject 1039 Model

In [107]:
torch.cuda.empty_cache()

In [108]:
subject_id = 1039

In [109]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.61s/it]


In [110]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.63s/it]


## Train Validation Split

In [111]:
dataset.train_validation_split(val_proportion=0.2)

In [112]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 224
Validation size: 56


## Run Training/Validation

In [113]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [114]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [115]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


In [116]:
 tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Subject 1043 Model

In [117]:
torch.cuda.empty_cache()

In [118]:
subject_id = 1043

In [119]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.37s/it]


In [120]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.06it/s]


## Train Validation Split

In [121]:
dataset.train_validation_split(val_proportion=0.2)

In [122]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 48
Validation size: 11


## Run Training/Validation

In [123]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [124]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [125]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


In [126]:
 tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Subject 1044 Model

In [127]:
torch.cuda.empty_cache()

In [128]:
subject_id = 1044

In [129]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.92s/it]


In [130]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.07it/s]


## Train Validation Split

In [131]:
dataset.train_validation_split(val_proportion=0.2)

In [132]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 68
Validation size: 16


## Run Training/Validation

In [133]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [134]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [135]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


In [136]:
 tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Subject 1048 Model

In [137]:
torch.cuda.empty_cache()

In [138]:
subject_id = 1048

In [139]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.51s/it]


In [140]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.26s/it]


## Train Validation Split

In [141]:
dataset.train_validation_split(val_proportion=0.2)

In [142]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 180
Validation size: 45


## Run Training/Validation

In [143]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [144]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [145]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


In [146]:
 tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Subject 1049 Model

In [147]:
torch.cuda.empty_cache()

In [148]:
subject_id = 1049

In [149]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.93s/it]


In [150]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.07it/s]


## Train Validation Split

In [151]:
dataset.train_validation_split(val_proportion=0.2)

In [152]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 125
Validation size: 31


## Run Training/Validation

In [153]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [154]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [155]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


In [156]:
 tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Subject 1051 Model

In [157]:
torch.cuda.empty_cache()

In [158]:
subject_id = 1051

In [159]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.54s/it]

1 samples with missing labels.
 Labels were replaced with key label mean.


In [160]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.32s/it]


## Train Validation Split

In [161]:
dataset.train_validation_split(val_proportion=0.2)

In [162]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 156
Validation size: 38


## Run Training/Validation

In [163]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [164]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [165]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


# Create REAL Dataset

In [166]:
############### REAL config ###############

dataset_name = 'REAL'
label_class = 'on_off'
sort_by = 'subject_id'
train_ts_dir = 'Data/BEATPDchallenge/BeatPDRealPDupdated/real-pd.training_data_updated/training_data/'
train_label_dir = 'Data/BEATPDchallenge/data_labels-realpd/REAL-PD_Training_Data_IDs_Labels.csv'
ancil_ts_dir = 'Data/BEATPDchallenge/BeatPDRealPDupdated/real-pd.ancillary_data_updated/ancillary_data/'
ancil_label_dir = 'Data/BEATPDchallenge/data_labels-realpd/REAL-PD_Ancillary_Data_IDs_Labels.csv'
test_ts_dir='Data/BEATPDchallenge/BeatPDRealPDupdated/real-pd.testing_data_updated/testing_data/'
test_data_id_dir='Data/BEATPDchallenge/REAL-PD_Test_Data_IDs.csv'

In [167]:
REALdataset = Dataset(dataset_name=dataset_name, sort_by=sort_by, label_class = label_class,
                           train_ts_dir=train_ts_dir, train_label_dir=train_label_dir,
                           ancil_ts_dir=ancil_ts_dir, ancil_label_dir=ancil_label_dir,
                           combine_ancil=False,test_ts_dir=test_ts_dir, test_data_id_dir=test_data_id_dir)

Creating Test Dictionary: 100%|██████████████████████████████████████████████████████| 191/191 [00:39<00:00,  4.88it/s]

Issue with 56 measurements.
 Enter self.issue_measuements for a list of them.


In [168]:
REALdataset.data_dict.keys()

dict_keys(['hbv013', 'hbv038', 'hbv017', 'hbv023', 'hbv051', 'hbv077', 'hbv054', 'hbv014', 'hbv018', 'hbv043', 'hbv022', 'hbv012'])

In [169]:
REALdataset.test_data_dict.keys()

dict_keys(['hbv013', 'hbv038', 'hbv017', 'hbv023', 'hbv051', 'hbv077', 'hbv054', 'hbv014', 'hbv018', 'hbv043', 'hbv022', 'hbv012'])

In [170]:
tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Subject hbv013 Model

In [171]:
torch.cuda.empty_cache()

In [172]:
subject_id = 'hbv013'

In [173]:
REALdataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.68s/it]

67 samples with missing labels.
 Labels were replaced with key label mean.


In [174]:
REALdataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.13s/it]


## Train Validation Split

In [175]:
REALdataset.train_validation_split(val_proportion=0.2)

In [176]:
print('Training size:',len(REALdataset.train_list))
print('Validation size:',len(REALdataset.val_list))

Training size: 161
Validation size: 40


## Run Training/Validation

In [177]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [178]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=REALdataset.train_list,
                       val_list=REALdataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [179]:
# Predict and fill in submission template
for spec, id_, subject, ts in REALdataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


In [180]:
tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Subject hbv038 Model

In [181]:
subject_id = 'hbv038'

In [182]:
REALdataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.78s/it]


In [183]:
REALdataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.42it/s]


## Train Validation Split

In [184]:
REALdataset.train_validation_split(val_proportion=0.2)

In [185]:
print('Training size:',len(REALdataset.train_list))
print('Validation size:',len(REALdataset.val_list))

Training size: 60
Validation size: 14


## Run Training/Validation

In [186]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [187]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=REALdataset.train_list,
                       val_list=REALdataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [188]:
# Predict and fill in submission template
for spec, id_, subject, ts in REALdataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


In [189]:
tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Subject hbv051 Model

In [190]:
torch.cuda.empty_cache()

In [191]:
subject_id = 'hbv051'

In [192]:
REALdataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.24s/it]

54 samples with missing labels.
 Labels were replaced with key label mean.


In [193]:
REALdataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.53it/s]


## Train Validation Split

In [194]:
REALdataset.train_validation_split(val_proportion=0.2)

In [195]:
print('Training size:',len(REALdataset.train_list))
print('Validation size:',len(REALdataset.val_list))

Training size: 130
Validation size: 32


## Run Training/Validation

In [196]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [197]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=REALdataset.train_list,
                       val_list=REALdataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [198]:
# Predict and fill in submission template
for spec, id_, subject, ts in REALdataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


In [199]:
tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Subject hbv077 Model

In [200]:
torch.cuda.empty_cache()

In [201]:
subject_id = 'hbv077'

In [202]:
REALdataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.51s/it]

58 samples with missing labels.
 Labels were replaced with key label mean.


In [203]:
REALdataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.39it/s]


## Train Validation Split

In [204]:
REALdataset.train_validation_split(val_proportion=0.2)

In [205]:
print('Training size:',len(REALdataset.train_list))
print('Validation size:',len(REALdataset.val_list))

Training size: 140
Validation size: 34


## Run Training/Validation

In [206]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [207]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=REALdataset.train_list,
                       val_list=REALdataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [208]:
# Predict and fill in submission template
for spec, id_, subject, ts in REALdataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


In [209]:
tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Subject hbv014 Model

In [210]:
torch.cuda.empty_cache()

In [211]:
subject_id = 'hbv014'

In [212]:
REALdataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.09s/it]

65 samples with missing labels.
 Labels were replaced with key label mean.


In [213]:
REALdataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.24it/s]


## Train Validation Split

In [214]:
REALdataset.train_validation_split(val_proportion=0.2)

In [215]:
print('Training size:',len(REALdataset.train_list))
print('Validation size:',len(REALdataset.val_list))

Training size: 156
Validation size: 39


## Run Training/Validation

In [216]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [217]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=REALdataset.train_list,
                       val_list=REALdataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [218]:
# Predict and fill in submission template
for spec, id_, subject, ts in REALdataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


In [219]:
tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Subject hbv043 Model

In [220]:
torch.cuda.empty_cache()

In [221]:
subject_id = 'hbv043'

In [222]:
REALdataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.93s/it]


In [223]:
REALdataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.42it/s]


## Train Validation Split

In [224]:
REALdataset.train_validation_split(val_proportion=0.2)

In [225]:
print('Training size:',len(REALdataset.train_list))
print('Validation size:',len(REALdataset.val_list))

Training size: 61
Validation size: 15


## Run Training/Validation

In [226]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [227]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=REALdataset.train_list,
                       val_list=REALdataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [228]:
# Predict and fill in submission template
for spec, id_, subject, ts in REALdataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


In [229]:
tst_sc1.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1039.0, 1043.0, 1044.0, 1048.0, 1049.0, 1051.0, 'hbv013',
       'hbv038', 'hbv051', 'hbv077', 'hbv014', 'hbv043', 'hbv022'],
      dtype=object)

# Subject hbv022 Model

In [230]:
torch.cuda.empty_cache()

In [231]:
subject_id = 'hbv022'

In [232]:
REALdataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.36s/it]

90 samples with missing labels.
 Labels were replaced with key label mean.


In [233]:
REALdataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.21it/s]


## Train Validation Split

In [234]:
REALdataset.train_validation_split(val_proportion=0.2)

In [235]:
print('Training size:',len(REALdataset.train_list))
print('Validation size:',len(REALdataset.val_list))

Training size: 216
Validation size: 54


## Run Training/Validation

In [236]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [237]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=REALdataset.train_list,
                       val_list=REALdataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [238]:
# Predict and fill in submission template
for spec, id_, subject, ts in REALdataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc1.index[tst_sc1['measurement_id'] == id_]
    tst_sc1.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torchvision.transforms.functional as TF


# Save Predictions

In [239]:
del tst_sc1['subject_id']

In [240]:
tst_sc1.prediction = tst_sc1.prediction.astype(int)

In [241]:
tst_sc1

,measurement_id,prediction
0,0e80673b-8c90-474f-8388-00fdf25795e4,0
1,a7b61d40-9f26-4878-a2d7-bd766b8ae6f5,0
2,fc851afd-133c-4e8f-8deb-0079cc442358,3
3,90c2c320-1007-47b4-9f8e-cdd37067dfb2,0
4,379fd23b-e38f-4214-8a70-7a829004359f,4
...,...,...
690,36e11b88-40f3-4ca3-81c7-f429a00960cd,0
691,f7360dd9-0477-481d-8cd5-024f4fc51d1f,0
692,f638f4b7-7fde-4b8d-8659-5e296b9508f6,0
693,34ec467b-aed9-4e73-8a95-17292119b8c5,0


In [244]:
#Save submission df to CSV
tst_sc1.to_csv('Data/on_off_predictions_final.csv', index=False)

In [245]:
# Check to see if csv saved correctly
pd.read_csv('Data/on_off_predictions_final.csv').head()

,measurement_id,prediction
0,0e80673b-8c90-474f-8388-00fdf25795e4,0
1,a7b61d40-9f26-4878-a2d7-bd766b8ae6f5,0
2,fc851afd-133c-4e8f-8deb-0079cc442358,3
3,90c2c320-1007-47b4-9f8e-cdd37067dfb2,0
4,379fd23b-e38f-4214-8a70-7a829004359f,4
